# Predict with LoRA

In [29]:
from model_utils_bert import BertRnn, BertRnnDist
from transformers import Trainer, TrainingArguments, BertConfig
from transformers import  AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from dataloader_bert import DataSetLoaderBERT
from utils import compute_metrics
import json
from transformers import TextClassificationPipeline
import torch
from transformers import set_seed
set_seed(10)

Load model and tokenizer

In [30]:
model_name = "/M2/ArgosMHC_models/checkpoints/lora_t33_c3_2/checkpoint-150000/"  # mejor checkpoiunt
name_results = "predictions_esm2_lora_t33_c3" # 
pre_trained = "/M2/ArgosMHC_models/pre_trained_models/esm2_t33_650M_UR50D/"

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(pre_trained)

#In case you have added tokens, it’s recommended to use the PeftModel class rather than AutoModelForCausalLM. The former takes into account resizing the embedding matrix.

Some weights of the model checkpoint at /M2/ArgosMHC_models/checkpoints/lora_t33_c3_2/checkpoint-150000/ were not used when initializing BertForSequenceClassification: ['base_model.model.bert.encoder.layer.19.attention.output.LayerNorm.bias', 'base_model.model.bert.encoder.layer.26.attention.self.value.lora_A.default.weight', 'base_model.model.bert.encoder.layer.25.attention.self.query.base_layer.weight', 'base_model.model.bert.encoder.layer.14.attention.output.LayerNorm.bias', 'base_model.model.bert.encoder.layer.23.attention.output.dense.weight', 'base_model.model.bert.encoder.layer.12.attention.output.LayerNorm.bias', 'base_model.model.bert.encoder.layer.4.attention.self.query.lora_B.default.weight', 'base_model.model.bert.encoder.layer.16.attention.self.query.lora_A.default.weight', 'base_model.model.bert.encoder.layer.12.attention.self.value.lora_A.default.weight', 'base_model.model.bert.encoder.layer.21.intermediate.dense.weight', 'base_model.model.bert.encoder.layer.17.attention

Load dataset

In [32]:

seq_length = 50 # for MHC-I
test_dataset = DataSetLoaderBERT("/M2/ArgosMHC_models/dataset/hlab/hlab_test_micro.csv", tokenizer_name="/M2/ArgosMHC_models/pre_trained_models/esm2_t33_650M_UR50D", max_length=seq_length)

In [40]:
#for sample in test_dataset:
#    print(sample)
#sample = test_dataset[0]
#print(sample)
#print(sample['input_ids'])
#print( next(iter(test_dataset)) )

print( tokenizer("YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYLFGRDL", padding='max_length', max_length=seq_length) )
print(test_dataset[0])

print( tokenizer("YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYTDKKTHL", padding='max_length', max_length=seq_length) )
print(test_dataset[1])

#Genera estas diferencia cuando en el tokenizer no definimos que haga un padding 
""""
tokenizer 'input_ids': [0, 19, 18, 5, 20, 19, 16, 9, 17, 20, 5, 21, 11, 13, 5, 17, 11, 4, 19, 12, 12, 19, 10, 13, 19, 11, 22, 7, 5, 10, 7, 19, 10, 6, 19, 4, 18, 6, 10, 13, 4, 2],
dataloader'input_ids': [0, 19, 18, 5, 20, 19, 16, 9, 17, 20, 5, 21, 11, 13, 5, 17, 11, 4, 19, 12, 12, 19, 10, 13, 19, 11, 22, 7, 5, 10, 7, 19, 10, 6, 19, 4, 18, 6, 10, 13, 4, 2,  1,  1,  1,  1,  1,  1,  1,  1],

 
tokenizer 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
dataloader'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
"""


{'input_ids': [0, 19, 18, 5, 20, 19, 16, 9, 17, 20, 5, 21, 11, 13, 5, 17, 11, 4, 19, 12, 12, 19, 10, 13, 19, 11, 22, 7, 5, 10, 7, 19, 10, 6, 19, 4, 18, 6, 10, 13, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]}
{'input_ids': tensor([ 0, 19, 18,  5, 20, 19, 16,  9, 17, 20,  5, 21, 11, 13,  5, 17, 11,  4,
        19, 12, 12, 19, 10, 13, 19, 11, 22,  7,  5, 10,  7, 19, 10,  6, 19,  4,
        18,  6, 10, 13,  4,  2,  1,  1,  1,  1,  1,  1,  1,  1]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0]), 'labels': tensor(1)}
{'input_ids': [0, 19, 18, 5, 20, 19, 16, 9, 17, 20, 5, 21, 11, 13, 5, 17, 11, 4, 19, 12, 12, 19, 10, 13, 19, 11, 22, 7, 5, 10, 7, 19, 10, 6, 19, 11, 13, 15, 15, 11, 21, 4, 2, 1, 1, 

'"\ntokenizer \'input_ids\': [0, 19, 18, 5, 20, 19, 16, 9, 17, 20, 5, 21, 11, 13, 5, 17, 11, 4, 19, 12, 12, 19, 10, 13, 19, 11, 22, 7, 5, 10, 7, 19, 10, 6, 19, 4, 18, 6, 10, 13, 4, 2],\ndataloader\'input_ids\': [0, 19, 18, 5, 20, 19, 16, 9, 17, 20, 5, 21, 11, 13, 5, 17, 11, 4, 19, 12, 12, 19, 10, 13, 19, 11, 22, 7, 5, 10, 7, 19, 10, 6, 19, 4, 18, 6, 10, 13, 4, 2,  1,  1,  1,  1,  1,  1,  1,  1],\n\n \ntokenizer \'attention_mask\': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]\ndataloader\'attention_mask\': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]\n'

Pipeline

In [44]:
#tokenizer.padding='max_length'
#tokenizer.max_length=seq_length
print(tokenizer)
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True)
print(pipe("YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYLFGRDL")) # label 1
print(pipe("YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYTDKKTHL")) # label 1
print(pipe("YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYDCEKAFFKM")) # label 0
print(pipe("YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYASDDGSWWD")) # label 0

PreTrainedTokenizer(name_or_path='/M2/ArgosMHC_models/pre_trained_models/esm2_t33_650M_UR50D/', vocab_size=33, model_max_len=1000000000000000019884624838656, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<eos>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'cls_token': '<cls>', 'mask_token': '<mask>'})
[[{'label': 'LABEL_0', 'score': 0.3711804747581482}, {'label': 'LABEL_1', 'score': 0.628819465637207}]]
[[{'label': 'LABEL_0', 'score': 0.3667409121990204}, {'label': 'LABEL_1', 'score': 0.6332590579986572}]]
[[{'label': 'LABEL_0', 'score': 0.3783145546913147}, {'label': 'LABEL_1', 'score': 0.6216854453086853}]]
[[{'label': 'LABEL_0', 'score': 0.375162273645401}, {'label': 'LABEL_1', 'score': 0.6248377561569214}]]


# Predict Classic

Este código, evalua el modelo despues del fine-tuning en el conjunto de testing.

In [65]:
from model_utils_bert import BertRnn, BertRnnDist
from transformers import Trainer, TrainingArguments, BertConfig
from transformers import  AutoModel, AutoTokenizer, AutoModelForSequenceClassification
from dataloader_bert import DataSetLoaderBERT
from utils import compute_metrics
import json
from transformers import TextClassificationPipeline, pipeline, BartForSequenceClassification
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import set_seed
import pandas as pd
import numpy as np

set_seed(10)

Abrimos el modelo y el tokenizer. Tambien definimos la ruta del modelo, el tokenizer (modelo pre entrenado) y la base de datos de testing. Finalmente, definimos el sufijo de de los archivos a generar como salida.

In [ ]:
model_name = "/M2/ArgosMHC_models/checkpoints/classic_t6_c3/checkpoint-102000/"  # mejor checkpoiunt
name_results = "predictions_esm2_t33_c5" # 
#pre_trained = "/M2/ArgosMHC_models/pre_trained_models/esm2_t33_650M_UR50D/"
pre_trained = "/M2/ArgosMHC_models/pre_trained_models/esm2_t6_8M_UR50D/"
dataset = "/M2/ArgosMHC_models/dataset/hlab/hlab_test_micro.csv"

model = BertRnn.from_pretrained(model_name, num_labels=2) # it fail for automodel for sequence classification
tokenizer = AutoTokenizer.from_pretrained(pre_trained)

Definimos el DataLoader

In [47]:
seq_length = 50 # for MHC-I
test_dataset = DataSetLoaderBERT(dataset, tokenizer_name=pre_trained, max_length=seq_length)
data_iter = DataLoader(test_dataset, batch_size=16, shuffle=False)

print( tokenizer("YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYLFGRDL", padding='max_length', max_length=seq_length) )
print(test_dataset[0])
print(type(test_dataset[0]))

loading file vocab.txt
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json


{'input_ids': [0, 19, 18, 5, 20, 19, 16, 9, 17, 20, 5, 21, 11, 13, 5, 17, 11, 4, 19, 12, 12, 19, 10, 13, 19, 11, 22, 7, 5, 10, 7, 19, 10, 6, 19, 4, 18, 6, 10, 13, 4, 2, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]}
{'input_ids': tensor([ 0, 19, 18,  5, 20, 19, 16,  9, 17, 20,  5, 21, 11, 13,  5, 17, 11,  4,
        19, 12, 12, 19, 10, 13, 19, 11, 22,  7,  5, 10,  7, 19, 10,  6, 19,  4,
        18,  6, 10, 13,  4,  2,  1,  1,  1,  1,  1,  1,  1,  1]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0]), 'labels': tensor(1)}
<class 'dict'>


Prediccion usando el dataloader y según el batch size

In [84]:
# data_iter, es un dataLoader, de la base de datos de test, y tiene un batch_size de 16
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

model.eval() # is a kind of switch for some specific layers/parts of the model that behave differently during training and inference (evaluating) time. For example, Dropouts Layers, BatchNorm Layers etc. You need to turn them off during model evaluation,

predictions = []
labels = []

with torch.no_grad(): # turn off gradients computation
    for i, batch in enumerate(data_iter): # por cada batch        
        labels.extend(batch['labels'].numpy())
        output = model(batch['input_ids'], batch['attention_mask']) # inference
        for row_sample in output.logits: # por cada muestra del batch
            logits = row_sample.numpy()
            probs = softmax(logits)
            predictions.append( [logits[0], logits[1], probs[0], probs[1]] )

#print(predictions)
df = pd.DataFrame(predictions, columns=["logit_class_0", "logit_class_1", "prob_class_0", "prob_class_0"])
df['prediction'] = df.apply(lambda row: ( 0 if row[0] > row[1] else 1 ), axis=1)
df['label'] = labels
print(df)

    logit_class_0  logit_class_1  prob_class_0  prob_class_0  prediction  \
0        1.484725      -2.111731      0.973311      0.026689           0   
1        0.366563      -0.806652      0.763726      0.236274           0   
2       -3.797246       3.086962      0.001023      0.998977           1   
3       -4.328579       3.746734      0.000311      0.999689           1   
4       -1.433432       0.912185      0.087415      0.912585           1   
..            ...            ...           ...           ...         ...   
71       1.846199      -2.293438      0.984321      0.015679           0   
72      -0.228940      -0.585798      0.588280      0.411720           0   
73       1.457081      -2.315464      0.977523      0.022477           0   
74       1.226935      -1.907617      0.958296      0.041704           0   
75       2.441628      -3.086117      0.996041      0.003959           0   

    label  
0       1  
1       1  
2       1  
3       1  
4       1  
..    ...  
71 

Ejemplo de como hacer la Prediccion usando una sola muestra

In [73]:
# algunas muestras
"YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYRSDTPLIY" # label 1
"YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYNSDLVQKY" # label 1
"YFAMYGEKVAHTHVDTLYLRYHYYTWAVWAYTWYLLAASEAPR"  # label 0
"YFAMYGEKVAHTHVDTLYLRYHYYTWAVWAYTWYQWSEKVTEE"  # label 0

# el tokenizer devuelve los input_ids y el attention_mask como dos listas
sample = tokenizer("YFAMYQENMAHTDANTLYIIYRDYTWVARVYRGYNSDLVQKY", padding='max_length', max_length=seq_length)

# convertimos en tensor, debe ser lista de listas
ids = torch.IntTensor([sample['input_ids']]) # tensor 2D
masks = torch.IntTensor([sample['attention_mask']]) # tensor 2D

model.eval()
with torch.no_grad(): # turn off gradients computation
    output = model(ids, masks )   

print("output", output)

output SequenceClassifierOutput(loss=None, logits=tensor([-4.3286,  3.7467]), hidden_states=None, attentions=None)


# Predict classic with Trainer

Este ejemplo, tambien hace las predicciones y cálcula las métricas, pero usando el Trainner.

In [91]:
# Predictions for HLAB dataset for TAPE

# load model
from model_utils_bert import BertRnn, BertRnnDist
from transformers import Trainer, TrainingArguments, BertConfig
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, roc_auc_score
from dataloader_bert import DataSetLoaderBERT
from transformers import BertConfig
from utils import compute_metrics
import json

model_name = "/M2/ArgosMHC_models/checkpoints/classic_t6_c3/checkpoint-102000/"  # mejor checkpoiunt
pre_trained = "/M2/ArgosMHC_models/pre_trained_models/esm2_t6_8M_UR50D/"
dataset = "/M2/ArgosMHC_models/dataset/hlab/hlab_test_micro.csv"

seq_length = 50 # for MHC-I
config = BertConfig.from_pretrained(model_name, num_labels=2 )

model = Trainer(model = BertRnn.from_pretrained(model_name, config=config), compute_metrics = compute_metrics)
test_dataset = DataSetLoaderBERT(dataset, tokenizer_name=pre_trained, max_length=seq_length)
predictions, label_ids, metrics = model.predict(test_dataset)

df = pd.DataFrame(predictions)

df['prediction'] = df.apply(lambda row: ( 0 if row[0] > row[1] else 1 ), axis=1)
df['label'] = label_ids
print(df)
#df.to_csv(name_results + ".csv")

#print(predictions)
print(metrics)

loading configuration file /M2/ArgosMHC_models/checkpoints/classic_t6_c3/checkpoint-102000/config.json
Model config BertConfig {
  "_name_or_path": "../pre_trained_models/esm2_t6_8M_UR50D",
  "architectures": [
    "BertRnn"
  ],
  "attention_probs_dropout_prob": 0.0,
  "classifier_dropout": null,
  "cnn_dropout": 0.1,
  "cnn_filters": 512,
  "emb_layer_norm_before": false,
  "esmfold_config": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 320,
  "initializer_range": 0.02,
  "intermediate_size": 1280,
  "is_folding_model": false,
  "layer_norm_eps": 1e-05,
  "length": 50,
  "mask_token_id": 32,
  "max_position_embeddings": 1026,
  "model_type": "bert",
  "num_attention_heads": 20,
  "num_hidden_layers": 6,
  "num_rnn_layer": 2,
  "pad_token_id": 1,
  "position_embedding_type": "rotary",
  "rnn": "lstm",
  "rnn_dropout": 0.1,
  "rnn_hidden": 768,
  "token_dropout": true,
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,

           0         1  prediction  label
0   1.484571 -2.111625           0      1
1   0.366120 -0.806223           0      1
2  -3.797319  3.086969           1      1
3  -4.328716  3.746843           1      1
4  -1.433371  0.912052           1      1
..       ...       ...         ...    ...
71  1.846014 -2.293381           0      0
72 -0.229090 -0.585730           0      0
73  1.457083 -2.315473           0      0
74  1.226967 -1.907715           0      0
75  2.441543 -3.086087           0      0

[76 rows x 4 columns]
{'test_loss': 0.3903467059135437, 'test_auc': 0.8815789473684211, 'test_precision': 0.9393939393939394, 'test_recall': 0.8157894736842105, 'test_f1': 0.8732394366197183, 'test_sn': 0.9393939393939394, 'test_sp': 0.9473684210526315, 'test_accuracy': 0.881578947368421, 'test_mcc': 0.7698512161427534, 'test_runtime': 0.1327, 'test_samples_per_second': 572.638, 'test_steps_per_second': 75.347}
